# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096931


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:40,  1.45s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:49,  1.82s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:34,  1.33s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:35,  1.41s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.60it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.01it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  5.01it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.55it/s]

Jnursssmith5263                       0.002493
Alexander00                           0.000410
ellivc                                0.001117
TateK                                 0.002411
jnarayanbvg                           0.001227
awright5                              0.005742
not-logged-in-dd0123951976ca40de3b    0.000649
framosuw                              0.020068
jaksonA                               0.053612
not-logged-in-5a8bb7be433678ade709    0.001268
KjellN                                0.001189
not-logged-in-7cc17a8a1e6ff5a23d80    0.003688
not-logged-in-86a7868b6f53133e9a58    0.011896
not-logged-in-468edef11712cfc1e24c    0.005035
not-logged-in-5f30b2bb9b7f9d49da41    0.143709
coyotes54                             0.000481
not-logged-in-314d48142a3a122ba666    0.003129
Giantjack14                           0.347905
not-logged-in-9f30f411df9d28d2c541    0.002355
BraxtonKinney                         0.001011
acapirala                             0.004318
Mr_DMan      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())